## Сервис для чтения книг  

Цель проекта  - проанализировать базу данных и ответить на вопросы :  

- Посчитайте, сколько книг вышло после 1 января 2000 года;  
- Для каждой книги посчитайте количество обзоров и среднюю оценку;  
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;  
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;  
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок;  

### Подключение к БД

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Загрузка данных

In [3]:
query_books = ''' SELECT *
        FROM books
        LIMIT 10
    '''
books = pd.io.sql.read_sql(query_books, con = engine)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


In [4]:
query_authors = ''' SELECT *
        FROM authors
        LIMIT 10
    '''
authors = pd.io.sql.read_sql(query_authors, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


In [5]:
query_ratings = ''' SELECT *
        FROM ratings
        LIMIT 10
    '''
ratings = pd.io.sql.read_sql(query_ratings, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


In [6]:
query_reviews = ''' SELECT *
        FROM reviews
        LIMIT 10
    '''
reviews = pd.io.sql.read_sql(query_reviews, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


In [7]:
query_publishers = ''' SELECT *
        FROM publishers
        LIMIT 10
    '''
publishers = pd.io.sql.read_sql(query_publishers, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


### Запросы под задачи

In [8]:
# Количество книг выпущенных после 1 января 2000 года
books_count = '''SELECT 
COUNT (book_id) AS cnt
FROM books
WHERE publication_date > '2000-01-01'
'''
books_count = pd.io.sql.read_sql(books_count, con = engine)
books_count

,cnt
0,819


После 1 января 2000 года было выпущено 819 книг

In [9]:
# Найдем для каждой книги количество обзоров и среднюю оценку
reviews_ratings = '''SELECT
        ratings.book_id AS book_id,
        COUNT(reviews.text) AS total_reviews,
        AVG(ratings.rating) AS mean_rating
        FROM
            ratings
        LEFT JOIN reviews ON 
            (reviews.book_id = ratings.book_id
            AND reviews.username = ratings.username)
        GROUP BY 
            ratings.book_id
        ORDER BY 
            ratings.book_id
'''
reviews_ratings = pd.io.sql.read_sql(reviews_ratings, con = engine)
reviews_ratings

,book_id,total_reviews,mean_rating
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
...,...,...,...
995,996,3,3.666667
996,997,3,3.400000
997,998,4,3.200000
998,999,2,4.500000


In [10]:
#Найдем издательство с наибольшим количеством книг с количеством страниц от 50 
publishers = '''SELECT
        publishers.publisher AS publisher,
        COUNT(books.book_id) AS total_books
        FROM
            books
        LEFT JOIN publishers ON 
            publishers.publisher_id = books.publisher_id
        GROUP BY 
            publishers.publisher
        HAVING
            SUM(books.num_pages) > 50
        ORDER BY 
            total_books DESC
        LIMIT 5
'''
publishers = pd.io.sql.read_sql(publishers, con = engine)
publishers

,publisher,total_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


С наибольшим количеством книг Penguin Books	- 42 книги. За ним в порядке убывания количества книг :  
Vintage - 31 книга  
Grand Central Publishing - 25 книг  
Penguin Classics - 24 книги  
Bantam - 19 книг  

In [11]:
# Найдем автора с самой высокой средней оценкой , учитывая книги с количеством оценок от 50
author_rate = '''SELECT
    authors.author,
    AVG(filtered.average_rating) AS avg_rating
    FROM
        (SELECT
            book_id AS book_id,
            COUNT(rating_id) AS rating_counts,
            AVG(rating) AS average_rating
        FROM
            ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) >= 50) 
        AS filtered
    INNER JOIN books ON books.book_id = filtered.book_id
    INNER JOIN authors ON authors.author_id = books.author_id
    GROUP BY authors.author
    ORDER BY avg_rating DESC
    LIMIT 10
'''
author_rate = pd.io.sql.read_sql(author_rate, con = engine)
author_rate


,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой - 4,28

In [12]:
# найдем среднее количество обзоров от пользоватолей, поставивших более 50 оценок
mean_reviews = '''
        SELECT
        AVG(sub.count)
        FROM
        (SELECT COUNT(*)
            FROM reviews
            WHERE username IN
                (SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating) > 50)
            GROUP BY username)
            AS sub;
        '''

reviews = pd.io.sql.read_sql(mean_reviews, con = engine)
reviews

,avg
0,24.333333


В среднем пользователи, поставившие более 50 оценок  оставляют примерно 24 рецензии

### Общий вывод   
Из наших данных получили следующие ответы на вопросы:  
- количество книг выпущенных после 1 января 2000 года - 819  
- издательство Penguin Books - лидер по выпущенному количеству книг с более чем 50 страницами  
- Джоан Роулинг -  самый высокооценный автор в наших данных
- Пользователи, поставившие более 50 оценок, в среднем оставляют 24 рецензии